# Finetuning Notebook
## LLM finetuning for AES (PERSUADE 2.0)

Code to finetune models for AES tasks. This was run on Google Colab to finetune Gemma 3 12B, as local resources were not enough. It is included here but not tested on the local environment due to the previously mentioned reason. Some changes might have to be made to accommodate setups.

### Force reinstall of libraries:

(Google Colab is weird)

In [ ]:
!pip install bitsandbytes
!pip install fsspec
!pip install -U datasets

  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2025.3.0-py3-none-any.whl (193 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.3/515.3 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.7.0
    Uninstalling fsspec-2025.7.0:
      Successfully uninstalled fsspec-2025.7.0
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.23.5
    Uninstalling huggingface-hub-0.23.5:
      Successfully uninstalled huggingface-hub-0.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.7.1+cu128 requires torch==2.7.1, but you have torch 2.2.2+cpu which is incompatible.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [ ]:
!pip install -U peft
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 110.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 78.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.3
    Uninstalling transformers-4.46.3:
      Successfully uninstalled transformers-4.46.3


In [ ]:
!pip install --force-reinstall flash-attn

  Using cached flash_attn-2.8.1-cp311-cp311-linux_x86_64.whl
  Using cached torch-2.7.1-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (29 kB)
  Using cached einops-0.8.1-py3-none-any.whl.metadata (13 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached typing_extensions-4.14.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.7.0-py3-none-any.whl.metadata (12 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.6.77-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.6.77-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.6.80-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.5.1.17-py3-none-manylinux_

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128

Looking in indexes: https://download.pytorch.org/whl/cu128
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 80.9 MB/s eta 0:00:00
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.21.0+cu124
    Uninstalling torchvision-0.21.0+cu124:
      Successfully uninstalled torchvision-0.21.0+cu124
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.6.0+cu124
    Uninstalling torchaudio-2.6.0+cu124:
      Successfully uninstalled torchaudio-2.6.0+cu124
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This b

### Create Slim Version of test/train set:

Remove metadata and just leave scores with texts

In [ ]:
import pandas as pd, os

name_dir     = "data"
file_name_train = "persuade_corpus_2.0_train.csv"
file_name_test  = "persuade_corpus_2.0_test.csv"
slim_columns    = ["essay_id_comp", "full_text", "holistic_essay_score"]

for split, fname in [("train", file_name_train), ("test", file_name_test)]:
    fullpath = os.path.join(name_dir, fname)
    print(f"loading {split} from {fullpath} …")
    df = pd.read_csv(fullpath)

    slim = df[slim_columns].copy()
    outpath = os.path.join(name_dir, f"slim_persuade_{split}.csv")
    print(f"writing slim {split} → {outpath}")
    slim.to_csv(outpath, index=False)

loading train from data/persuade_corpus_2.0_train.csv …


/tmp/ipython-input-4-2338030883.py:11: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fullpath)


writing slim train → data/slim_persuade_train.csv
loading test from data/persuade_corpus_2.0_test.csv …


/tmp/ipython-input-4-2338030883.py:11: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fullpath)


writing slim test → data/slim_persuade_test.csv


#### Login to HuggingFace Hub:

In [ ]:
from huggingface_hub import login
login()

#### Tokenise and preprocess dataset:

In [ ]:
import os

# to avoid out of memory
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True,max_split_size_mb:128"

from datasets import load_dataset
from transformers import AutoTokenizer

# paths
train_path = "data/slim_persuade_train.csv"
valid_path = "data/slim_persuade_test.csv"

raw_ds = load_dataset(
    "csv",
    data_files={"train": train_path, "valid": valid_path},
)

llm_name = "google/gemma-3-12b-it"
tokenizer = AutoTokenizer.from_pretrained(llm_name, use_fast=True)
tokenizer.pad_token_id = tokenizer.eos_token_id

# constants
TEXT_COL = "full_text"
SCORE_COL = "holistic_essay_score"
MAX_LEN = 1024 # tokens limit

PROMPT_PREFIX = (
    "You are an essay rater specializing in the evaluation of essays written by students "
    "from 6th to 12th grade. Read and evaluate the essay. Assign it a score from 1 to 6, "
    "in increments of 1. Your response should be only a numeric value representing the score you gave."
)

def build_prompt_only(text: str) -> str:
    # prompt without score, for inference
    return f"<s>[INST] {PROMPT_PREFIX} [/INST]\n{text.strip()}\n</s>"

def build_prompt_with_score(text: str, score: int) -> str:
    # prompt with score, for training
    return f"<s>[INST] {PROMPT_PREFIX} [/INST]\n{text.strip()}\n</s> {score}"

def preprocess(batch):
    texts  = batch[TEXT_COL]
    scores = batch[SCORE_COL]

    # normalise scores between 1 to 6 and cast to int
    norm_scores = []
    for s in scores:
        try:
            s_int = int(round(float(s)))
        except Exception:
            # fail is 1
            s_int = 1
        s_int = min(max(s_int, 1), 6)
        norm_scores.append(s_int)

    # build full supervised strings
    full_strs = [build_prompt_with_score(t, s) for t, s in zip(texts, norm_scores)]

    # tokenise
    tokenized = tokenizer(
        full_strs,
        truncation=True,
        max_length=MAX_LEN,
        padding=False,
    )

    # tokenise answers separately to know label length
    ans_tok = tokenizer(
        [str(s) for s in norm_scores],
        add_special_tokens=False,
    )["input_ids"]  # list of list[int]

    # build label masks: -100 for everything except the final answer tokens
    labels = []
    input_ids = tokenized["input_ids"]
    for ids, ans_ids in zip(input_ids, ans_tok):
        # protect against truncation: if answer got cut off, drop example or mask all -100
        if len(ids) < len(ans_ids):
            labels.append([-100] * len(ids))
            continue
        # assume answer tokens are at the end (because we appended them last)
        start = len(ids) - len(ans_ids)
        lab = [-100] * start + ids[start:]
        labels.append(lab)

    tokenized["labels"] = labels
    tokenized["gold_score"] = norm_scores

    return tokenized

ds = raw_ds.map(
    preprocess,
    batched=True,
    remove_columns=raw_ds["train"].column_names,
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating valid split: 0 examples [00:00, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

Map:   0%|          | 0/173266 [00:00<?, ? examples/s]

Map:   0%|          | 0/112117 [00:00<?, ? examples/s]

#### Load models and prepare lora adapter, quantization:

In [ ]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from transformers.utils import is_flash_attn_2_available
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# custom collator to preserve masked labels
def aes_data_collator(features):
    input_ids      = [f["input_ids"] for f in features]
    attention_mask = [f["attention_mask"] for f in features]
    labels_list    = [f["labels"] for f in features]

    batch = tokenizer.pad(
        {"input_ids": input_ids, "attention_mask": attention_mask},
        padding=True,
        pad_to_multiple_of=8,
        return_tensors="pt",
    )
    max_len = batch["input_ids"].shape[1]
    padded_labels = []
    for lab in labels_list:
        if len(lab) < max_len:
            lab = lab + [-100] * (max_len - len(lab))
        padded_labels.append(lab[:max_len])
    batch["labels"] = torch.tensor(padded_labels, dtype=torch.long)
    return batch

# quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

attn_impl = "flash_attention_2" if is_flash_attn_2_available() else "sdpa"

# load base model
model = AutoModelForCausalLM.from_pretrained(
    llm_name,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    attn_implementation=attn_impl,
    device_map="auto",
    offload_folder="/tmp/gemma3_offload",
    offload_state_dict=True
)

# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj","v_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)

model = prepare_model_for_kbit_training(model)

# gradient checkpointing
model.gradient_checkpointing_enable()
model.enable_input_require_grads()
model.config.use_cache = False

# attach lora
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

trainable params: 12,607,488 || all params: 12,199,932,528 || trainable%: 0.1033


#### Main training loop:

In [ ]:
from transformers import Trainer, TrainingArguments, TrainerCallback
import time, torch

torch.cuda.empty_cache()

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/persuade_checkpoints",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    eval_accumulation_steps=1,
    learning_rate=3e-4,
    bf16=True,
    fp16=False,
    optim="paged_adamw_8bit",
    logging_steps=50,
    save_steps=50,
    save_total_limit=3,
    max_steps=2000,
    eval_strategy="no",
    remove_unused_columns=True,
    max_grad_norm=0.3,
    gradient_checkpointing=True,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],
    # eval_dataset=ds["valid"],  # seems to throw oom
    tokenizer=tokenizer,
    data_collator=aes_data_collator,
)

class StepLogger(TrainerCallback):
    def on_step_end(self, args, state, control, **kwargs):
        if state.global_step % 100 == 0:
            print(f"[{time.strftime('%H:%M:%S')}] Completed step {state.global_step}")

trainer.add_callback(StepLogger())
trainer.train()

/tmp/ipython-input-5-3765872386.py:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
You're using a GemmaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.bfloat16.


Step,Training Loss
50,21.877800
100,11.660700
150,9.063600
200,8.234800
250,9.424700
300,8.373400
350,8.840100
400,8.036600
450,8.020800
500,7.580800


[20:23:43] Completed step 100
[20:36:52] Completed step 200
[20:50:00] Completed step 300
[21:03:01] Completed step 400
[21:15:58] Completed step 500
[21:28:54] Completed step 600
[21:41:49] Completed step 700
[21:54:44] Completed step 800
[22:07:38] Completed step 900
[22:20:38] Completed step 1000
[22:33:33] Completed step 1100
[22:46:29] Completed step 1200
[22:59:25] Completed step 1300
[23:12:22] Completed step 1400
[23:25:15] Completed step 1500
[23:38:10] Completed step 1600
[23:51:07] Completed step 1700
[00:04:26] Completed step 1800
[00:17:43] Completed step 1900
[00:30:50] Completed step 2000


TrainOutput(global_step=2000, training_loss=7.05435555267334, metrics={'train_runtime': 15616.2249, 'train_samples_per_second': 1.025, 'train_steps_per_second': 0.128, 'total_flos': 6.50856070799871e+17, 'train_loss': 7.05435555267334, 'epoch': 0.09234356423072039})

#### Check for Torch and Flash Attention

In [ ]:
import torch
import flash_attn
from transformers.utils import is_flash_attn_2_available

print("Torch:", torch.__version__, torch.version.cuda)
print("flash-attn:", flash_attn.__version__)
print("flash-attn OK?", is_flash_attn_2_available())

Torch: 2.7.1+cu126 12.6
flash-attn: 2.8.1
flash-attn OK? True


### Inference with finetuned model

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

BASE = "google/gemma-3-12b-it"
ADAPT = "/content/drive/MyDrive/persuade_checkpoints/checkpoint-2000"

tokenizer = AutoTokenizer.from_pretrained(BASE, use_fast=True)
tokenizer.pad_token_id = tokenizer.eos_token_id

bnb = BitsAndBytesConfig(
    load_in_4bit=True,               # fast + memory-light
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

base_model = AutoModelForCausalLM.from_pretrained(
    BASE,
    quantization_config=bnb,
    torch_dtype=torch.bfloat16,
    device_map="auto",               # puts layers on GPU 0
)

model = PeftModel.from_pretrained(base_model, ADAPT, device_map="auto")
model.eval()
model.config.use_cache = True        # speed for pure inference

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

BASE_ID   = "google/gemma-3-12b-it"
ADAPTER   = "/content/drive/MyDrive/persuade_checkpoints/checkpoint-2000"

bnb = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(BASE_ID, use_fast=True)
tokenizer.pad_token_id = tokenizer.eos_token_id

def load_model(base_only: bool):
    m = AutoModelForCausalLM.from_pretrained(
        BASE_ID,
        quantization_config=bnb,
        torch_dtype=torch.bfloat16,
        attn_implementation="flash_attention_2",
        device_map="auto",
    )
    if not base_only:
        m = PeftModel.from_pretrained(m, ADAPTER, device_map="auto")
    m.eval(); m.config.use_cache = True
    return m

base_model   = load_model(base_only=True)
tuned_model  = load_model(base_only=False)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
PROMPT_PREFIX = (
    "You are an essay rater specializing in the evaluation of essays written by students "
    "from 6th to 12th grade. Read and evaluate the essay. Assign it a score from 1 to 6, "
    "in increments of 1. Your response should be only a numeric value representing the score you gave."
)

SYSTEM = PROMPT_PREFIX

def prompt(txt): return f"<s>[INST] {SYSTEM} [/INST]\n{txt.strip()}\n</s>"

@torch.inference_mode()
def score(model, essay):
    toks = tokenizer(prompt(essay), return_tensors="pt").to(model.device)
    out  = model.generate(**toks, max_new_tokens=2, temperature=0.0001,
                          pad_token_id=tokenizer.eos_token_id,
                          eos_token_id=tokenizer.eos_token_id)
    ans  = tokenizer.decode(out[0][toks['input_ids'].shape[1]:],
                            skip_special_tokens=True).strip()
    return ans

In [ ]:
essay = "Should Phones Be Allowed in Schools? In today’s digital age, mobile phones have become an integral part of daily life, especially for students. While some argue that phones are a distraction in schools, others believe they offer valuable learning and safety benefits. The question of whether phones should be allowed in schools is complex, but a balanced approach that allows limited and regulated use seems most reasonable. One of the strongest arguments for allowing phones in schools is safety. In an emergency, a phone can be a lifeline between students and their families. Parents can quickly reach their children, and students can contact emergency services if needed. In this sense, having a phone provides peace of mind for both parents and students. Phones can also serve as useful educational tools. With the rise of online learning platforms, apps, and digital resources, students can use their devices to research topics, check assignments, or collaborate with classmates. Many schools now incorporate technology into their curriculum, and phones can complement this digital learning environment if used responsibly. However, the downside cannot be ignored. Phones can be a major distraction during class. Texting, gaming, and social media can easily pull students’ attention away from lessons. This not only affects their learning but can also disrupt the classroom environment. Inappropriate use, such as cheating on tests or cyberbullying, is another concern that schools must take seriously. To address these issues, schools should adopt clear policies. For example, phones could be allowed only during break times or used in class under teacher supervision. Some schools use ‘phone lockers’ or require students to place phones in designated areas during lessons. These solutions maintain access while limiting distractions. Ultimately, banning phones outright may be too extreme, considering their importance in modern life. Instead, educating students on responsible phone use and setting clear boundaries offers a more effective and realistic approach. When used wisely, phones can support learning and communication without overwhelming the classroom. In conclusion, while phones can pose challenges in schools, they also offer undeniable benefits. Rather than enforce a strict ban, schools should focus on managing phone use through thoughtful policies and digital education. This approach respects the role technology plays in students’ lives while preserving the integrity of the learning environment."
print("Base Gemma :", score(base_model,  essay))
print("Fine-tuned :", score(tuned_model, essay))

Base Gemma : 
Fine-tuned : 4


In [ ]:
essay = "Mobile Phones in Schools: A Tool or a Trouble? The debate over mobile phones in schools has been ongoing for years. As technology continues to evolve, students are increasingly bringing phones into classrooms. This raises the question: should schools allow phones or ban them entirely? While there are valid concerns about distraction and misuse, phones can also offer significant educational and safety advantages when managed properly. One of the key benefits of phones in schools is improved communication. In cases of emergency or sudden schedule changes, students can quickly contact parents or guardians. This creates a safer and more responsive school environment. Additionally, phones can help students stay organized with calendar apps, reminders, and note-taking tools that assist them in managing homework and deadlines. From an educational standpoint, phones can enhance learning if used correctly. Students can access online resources, educational videos, and interactive apps to support classroom lessons. For instance, teachers might incorporate quizzes using mobile-based tools like Kahoot or Google Forms, turning passive learning into active participation. Moreover, in today’s tech-driven world, knowing how to use digital devices responsibly is a vital skill that schools should help cultivate. On the flip side, the potential for distraction is real. Phones can easily lead to off-task behavior like texting friends, browsing social media, or watching videos during class time. This not only affects the individual student’s performance but can also disrupt others. Teachers often struggle to keep students focused when phones are used irresponsibly. Cheating during exams and cyberbullying through social apps are additional problems that cannot be ignored. The best solution lies in balance. Instead of completely banning phones, schools should implement clear guidelines. For example, phones could be turned off or kept in backpacks during lessons, but allowed during breaks or specific learning activities. Teachers should have the authority to decide when phones are appropriate in class. Schools can also teach digital citizenship, helping students understand the responsibilities and consequences of their online behavior. In conclusion, while phones do present certain risks in the school environment, they also offer numerous benefits that shouldn’t be overlooked. With clear rules and guidance, schools can turn mobile phones from a source of distraction into a powerful tool for learning and safety. A well-regulated approach allows students to embrace technology while maintaining a focused and respectful classroom atmosphere"
print("Base Gemma :", score(base_model,  essay))
print("Fine-tuned :", score(tuned_model, essay))

Base Gemma : 
Fine-tuned : 5


In [ ]:
essay = "Why Blue Is the Best Color in the Rainbow. The rainbow is made up of seven beautiful colors: red, orange, yellow, green, blue, indigo, and violet. Each one has its charm, but among them, blue stands out as the best. It is not only visually calming and universally loved, but it also represents depth, peace, and intelligence. Blue has a unique ability to evoke emotion without being overwhelming. While red might symbolize passion and yellow may bring cheerfulness, blue brings a sense of calm and clarity. It is the color of the sky on a clear day and the vast oceans that cover most of our planet. This connection with nature makes blue a soothing and reassuring color in the rainbow. Scientifically, blue is also fascinating. It has one of the shortest wavelengths of visible light, which makes it scatter more easily in the atmosphere—hence why the sky appears blue. This natural presence all around us may be part of why it is so widely favored. Surveys around the world consistently show blue as the most popular color among people of all ages and cultures. Beyond aesthetics, blue carries powerful symbolic meanings. It often represents trust, wisdom, and intelligence. Think about the use of blue in logos for technology companies or healthcare—fields where reliability is crucial. It’s also a common color in uniforms and professional settings, reinforcing its association with responsibility and calm authority. In art and literature, blue has long been used to express depth of feeling and introspection. It can symbolize both sadness and serenity, making it one of the most emotionally versatile colors. While other colors in the rainbow have strong associations—like red with danger or green with growth—blue’s meanings are more balanced and nuanced. Of course, every color in the rainbow is important, and their combination creates a beautiful symbol of unity and diversity. But if one color had to be chosen as the best, blue makes a compelling case. It is peaceful without being dull, bold without being aggressive, and meaningful without being too obvious. In conclusion, blue’s natural beauty, emotional impact, cultural popularity, and symbolic depth make it the best color in the rainbow. Whether you’re gazing at the sky, diving into the sea, or simply admiring a rainbow after a storm, blue is the color that leaves a lasting impression"
print("Base Gemma :", score(base_model,  essay))
print("Fine-tuned :", score(tuned_model, essay))

Base Gemma : 6
Fine-tuned : 44
